In [37]:
import pandas as pd

In [38]:
pelis = pd.read_csv('pelis_combinado.csv')

In [39]:
pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557 entries, 0 to 1556
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nombre         1557 non-null   object
 1   Fecha          1557 non-null   object
 2   Tipo           1557 non-null   object
 3   Horario        1557 non-null   object
 4   FormatoImagen  1557 non-null   object
 5   FormatoIdioma  1557 non-null   object
 6   Cine           1557 non-null   object
dtypes: object(7)
memory usage: 85.3+ KB


In [40]:
pelis['Fecha'] = pd.to_datetime(pelis['Fecha'])
pelis.drop(columns=['Tipo'], inplace=True)

In [41]:
pelis.sort_values(by='Fecha', inplace=True)

pelis.set_index('Fecha', inplace=True)

In [42]:
pelis

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine
Fecha,,,,,
2025-03-08,ANORA,16:50,2D,Subtitulado,Cines del Centro
2025-03-08,flow,15:20,2D,Español,Las Tipas
2025-03-08,El Mono,22:50,2D,ESPAÑOL,Cinépolis
2025-03-08,flow,17:20,2D,Español,Las Tipas
2025-03-08,Attack on Titan: El Ataque Final,22:45,2D,SUBTITULADO,Cinépolis
...,...,...,...,...,...
2025-04-13,The chosen: la última cena,17:00,2D,Subtitulado,Showcase
2025-04-24,Pink Floyd at Pompeii – MCMLXXII,20:00,2D,Subtitulado,Showcase
2025-04-24,Pink Floyd at Pompeii – MCMLXXII,20:30,2D,SUBTITULADO,Cinépolis


---

In [43]:
print(pelis['Nombre'].unique())

['ANORA' 'flow' 'El Mono' 'Attack on Titan: El Ataque Final'
 'Mufasa: El Rey León' 'Los Pérez-Osos' 'Sonic 3: La Película' 'Cónclave'
 'Interestelar' 'Capitán América: Un Nuevo Mundo'
 'Bridget Jones: Loca por él' 'Mufasa: El rey león'
 'Bridget Jones: loca por él' 'Capitán América: Un nuevo mundo'
 'THE LAST ATTACK' 'Better Man: La historia de Robbie Williams'
 'Los Pérez-osos' 'Mickey 17' '1978' 'SONIC 3'
 'Gundam GQuuuuuuX: El Inicio' 'María Callas: En sus propias palabras'
 'Bambi: Una aventura en el bosque' 'Dog Man' 'Flow' 'Anora'
 'Yo antes de ti' 'Aún Estoy Aquí' 'Moana 2' 'Implacable' 'MARÍA CALLAS'
 'Red Velvet Happiness Diary: My Dear ReVe1uv En Cines'
 'Sonic 3: la película' 'Bambi: una aventura en el bosque' 'MICKEY 17'
 'Attack on titan: el ataque final' 'FLOW' 'LA PERSECUCIÓN'
 'Gundam GQuuuuuuX, El Inicio' 'Aún estoy aquí' 'CONCLAVE' 'El brutalista'
 'El mono' 'Better man: La historia de Robbie Williams' 'Septiembre 5'
 'Red Velvet Happiness Diary: My Dear, ReVe1uv En 

In [44]:
import re
from fuzzywuzzy import fuzz

def normalize(text):
    """
    Normalize the input string by converting to lowercase, 
    removing extra whitespaces, and stripping punctuation.
    """
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

def unify_movie_names(df, column_name='Nombre', similarity_threshold=80, new_col_name='Nombre_normalizado'):
    """
    Unify movie names in a DataFrame by creating a mapping of movie titles that are similar.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing movie names.
        column_name (str): The name of the column with the movie names to be normalized.
        similarity_threshold (int): The similarity threshold (0-100) used by fuzzywuzzy's ratio to classify
                                    two titles as equivalent. Default is 80.
        new_col_name (str): The name of the new column that will store the unified movie names.
    
    Returns:
        pd.DataFrame: The DataFrame with an additional column of unified movie names.
        dict: A mapping dictionary where each original movie title is mapped to its canonical title.
    """
    mapping = {}
    canonical_titles = []
    
    # Retrieve the unique titles in the given column
    unique_titles = df[column_name].unique()
    
    # Process each unique title
    for title in unique_titles:
        norm_title = normalize(title)
        matched = False
        # Compare with already-established canonical titles
        for canonical in canonical_titles:
            if fuzz.ratio(norm_title, normalize(canonical)) >= similarity_threshold:
                mapping[title] = canonical
                matched = True
                break
        if not matched:
            # If no similar title is found, this title becomes a new canonical title
            canonical_titles.append(title)
            mapping[title] = title
    
    # Map the canonical titles to the new column in the DataFrame
    df[new_col_name] = df[column_name].map(mapping)
    
    return df, mapping

    
   

In [45]:
# Apply the function
pelis, mapping = unify_movie_names(pelis)

print("Updated DataFrame:")
print(pelis)

print("\nMapping Dictionary:")
print(mapping)


Updated DataFrame:
                                      Nombre Horario FormatoImagen  \
Fecha                                                                
2025-03-08                             ANORA   16:50            2D   
2025-03-08                              flow   15:20            2D   
2025-03-08                           El Mono   22:50            2D   
2025-03-08                              flow   17:20            2D   
2025-03-08  Attack on Titan: El Ataque Final   22:45            2D   
...                                      ...     ...           ...   
2025-04-13        The chosen: la última cena   17:00            2D   
2025-04-24  Pink Floyd at Pompeii – MCMLXXII   20:00            2D   
2025-04-24  Pink Floyd at Pompeii – MCMLXXII   20:30            2D   
2025-04-27  Pink Floyd at Pompeii – MCMLXXII   17:00            2D   
2025-04-27  Pink Floyd at Pompeii – MCMLXXII   15:30            2D   

           FormatoIdioma              Cine                Nombre_norma

In [46]:
mapping

{'ANORA': 'ANORA',
 'flow': 'flow',
 'El Mono': 'El Mono',
 'Attack on Titan: El Ataque Final': 'Attack on Titan: El Ataque Final',
 'Mufasa: El Rey León': 'Mufasa: El Rey León',
 'Los Pérez-Osos': 'Los Pérez-Osos',
 'Sonic 3: La Película': 'Sonic 3: La Película',
 'Cónclave': 'Cónclave',
 'Interestelar': 'Interestelar',
 'Capitán América: Un Nuevo Mundo': 'Capitán América: Un Nuevo Mundo',
 'Bridget Jones: Loca por él': 'Bridget Jones: Loca por él',
 'Mufasa: El rey león': 'Mufasa: El Rey León',
 'Bridget Jones: loca por él': 'Bridget Jones: Loca por él',
 'Capitán América: Un nuevo mundo': 'Capitán América: Un Nuevo Mundo',
 'THE LAST ATTACK': 'THE LAST ATTACK',
 'Better Man: La historia de Robbie Williams': 'Better Man: La historia de Robbie Williams',
 'Los Pérez-osos': 'Los Pérez-Osos',
 'Mickey 17': 'Mickey 17',
 '1978': '1978',
 'SONIC 3': 'SONIC 3',
 'Gundam GQuuuuuuX: El Inicio': 'Gundam GQuuuuuuX: El Inicio',
 'María Callas: En sus propias palabras': 'María Callas: En sus pro

In [47]:
pelis.head(50)

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre_normalizado
Fecha,,,,,,
2025-03-08,ANORA,16:50,2D,Subtitulado,Cines del Centro,ANORA
2025-03-08,flow,15:20,2D,Español,Las Tipas,flow
2025-03-08,El Mono,22:50,2D,ESPAÑOL,Cinépolis,El Mono
2025-03-08,flow,17:20,2D,Español,Las Tipas,flow
2025-03-08,Attack on Titan: El Ataque Final,22:45,2D,SUBTITULADO,Cinépolis,Attack on Titan: El Ataque Final
2025-03-08,Mufasa: El Rey León,13:30,4D,Español,Las Tipas,Mufasa: El Rey León
2025-03-08,Mufasa: El Rey León,16:00,4D,Español,Las Tipas,Mufasa: El Rey León
2025-03-08,Los Pérez-Osos,17:20,2D,ESPAÑOL,Cinépolis,Los Pérez-Osos
2025-03-08,Sonic 3: La Película,20:20,2D,ESPAÑOL,Cinépolis,Sonic 3: La Película


In [48]:
pelis.drop(columns=['Nombre'], inplace=True)
pelis.rename(columns={'Nombre_normalizado': 'Nombre'}, inplace=True)

---

In [49]:
def convert_caps_to_title(text):
    """
    Convert any word in all CAPS to a version with only the first letter capitalized.
    
    Parameters:
        text (str): The input string to process.
        
    Returns:
        str: The processed string with all-CAPS words converted.
    """
    def replacer(match):
        word = match.group(0)
        # Check if the entire word is uppercase (this works with Unicode letters as well)
        if word.isupper():
            return word.capitalize()
        return word  # Return unchanged if not fully uppercase
    
    # This regex will match any word composed of word characters.
    # The re.UNICODE flag ensures Unicode characters are considered word characters.
    return re.sub(r'\b\w+\b', replacer, text, flags=re.UNICODE)


def normalize_formato_idioma(df, column_name='FormatoIdioma'):
    """
    Normalizes the words in CAPs within the given column of the DataFrame so that 
    they use only the first letter in caps.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the column to process.
        column_name (str): The name of the column to apply the normalization on.
    
    Returns:
        pd.DataFrame: The DataFrame with the updated column.
    """
    # Ensure we work with string data by converting the column values
    df[column_name] = df[column_name].astype(str).apply(convert_caps_to_title)
    return df


In [50]:
# Apply the normalization function on the 'FormatoIdioma' column
pelis = normalize_formato_idioma(pelis, column_name='FormatoIdioma')

In [51]:
pelis.head(50)

,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre
Fecha,,,,,
2025-03-08,16:50,2D,Subtitulado,Cines del Centro,ANORA
2025-03-08,15:20,2D,Español,Las Tipas,flow
2025-03-08,22:50,2D,Español,Cinépolis,El Mono
2025-03-08,17:20,2D,Español,Las Tipas,flow
2025-03-08,22:45,2D,Subtitulado,Cinépolis,Attack on Titan: El Ataque Final
2025-03-08,13:30,4D,Español,Las Tipas,Mufasa: El Rey León
2025-03-08,16:00,4D,Español,Las Tipas,Mufasa: El Rey León
2025-03-08,17:20,2D,Español,Cinépolis,Los Pérez-Osos
2025-03-08,20:20,2D,Español,Cinépolis,Sonic 3: La Película


---

In [53]:
pelis.to_csv('pelis_combinado_clean.csv', index=True)